<a href="https://colab.research.google.com/github/joechanAU/fastai-work/blob/main/JC_FastAI_Course_Workbook_Chapter_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.2 MB/s eta 0:00:00


KeyboardInterrupt: ignored

In [ ]:
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

# Classification of MNIST Images Example


In [ ]:
# Get the MNIST sample dataset
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path

#path.ls()
#(path/'train').ls()

#torch.set_printoptions(threshold=10_000)


# make two lists, containing all the 3's and 7's in tensor format
three_tensors = [tensor(Image.open(o)) for o in (path/'train'/'3').ls().sorted()]
seven_tensors = [tensor(Image.open(o)) for o in (path/'train'/'7').ls().sorted()]

# combine (stack) the images into 3 dimensional tensors
stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_threes = torch.stack(three_tensors).float()/255


# Objective: Take a input image and output a classification of whether it is a 3 or a 7
# i.e: we use training images as our inputs and the 3 or 7 folder name as the target. We fit a
# equation to this, adjusting the weights using gradient descent and measuring improvement using
# a loss function

# Stated differently, we are representing an image as a array where is component is a number
# representing the value of a pixel.
# We already know the classification of each image (since this is training data) and thus we
# try to fit an equation which takes the image data (i.e: array) and outputs a value representing that
# classification.


# combine all the images into a single tensor. Each image is converted from a 28x28 matrix to a 1x784 vector
# pytorch view function changes the same of a tensor
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
train_y = tensor([1]*len(three_tensors) + [0]*len(seven_tensors)).unsqueeze(1)


# start with random weights
# we have a weight for each pixel (ie 784 weights)
def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

# weights = init_params((28*28,1))
# bias = init_params(1)

# preds = train_x @ weights + bias


# # We can (arbitarily) say that predictions > 0 represent a 3 and predictions <= 0 represent a 1
# # By comparing each of the predictions to the actual across the whole training set, we can calculate
# # an accuracy for this set of weights against the entire training set.

# corrects = (preds>0.0).float() == train_y
# accuracy = corrects.float().mean().item()
# # print("accuracy: ", accuracy)


# with torch.no_grad(): weights[0] *= 1.0001
# preds = train_x @ weights + bias
# corrects = (preds>0.0).float() == train_y
# accuracy = corrects.float().mean().item()
# #print("accuracy: ", accuracy)


# accuracy is a poor choice for our loss function as it is generally constant (0 or 1) everywhere except
# right at the threshold
# We need a loss function which accepts our predictions (not the actual images), the targets (i.e: the Y labels),
# and returns a better (lower) loss if the predictions are closer to the targets

# where the target is 1 (i.e.: the image is actually a 3) we return the distance between 1 and the prediction
# else the target must be 0 (i.e.: the image is a 7) so we return the distance between 0 and the prediction
#def minst_loss(predictions, targets):
#  return torch.where(targets==1, 1-predictions, predictions).mean()

# note however, that our predictions may not be between 0 and 1 (as the above loss function assumes)
# The Sigmoid function transforms any number in range -infinity to +infinity into the range 0 to 1.
# PyTorch includes an optimised version of Sigmoid
def minst_loss(predictions, targets):
  predictions = predictions.sigmoid()
  return torch.where(targets==1, 1-predictions, predictions).mean()




# Dataset stores the samples and their corresponding labels
# combine X and Y into a list of tuples ..
# zip function takes two iterators (x and y) and returns a separate iterator
# which returns tuples of the paired elements in x and y

train_dset = list(zip(train_x, train_y))

# Note on 'batches':
# We typically don't want to use the entire training set each epoch. Too few training examples
# will result in a poor fit while too many will be slow and potentially overload the GPU
# the DataLoader class provided by PyTorch/FastAI will randomise the training data and create mini-batches
dl = DataLoader(train_dset, batch_size=256, shuffle=True)

# lets define the model as a separate function:
def myModel(xb):
  return xb @ weights + bias

def printAccuracy(y_pred,y):
  corrects = (y_pred>0.0).float() == y
  accuracy = corrects.float().mean().item()
  print("accuracy: ", accuracy)


weights = init_params((28*28,1))
bias = init_params(1)

lr = 1


# for x,y in dl:

for i in range (50):

  x,y = next(iter(dl))
  preds = myModel(x)
  loss = minst_loss(preds, y)
  #print("loss: ", loss)
  loss.backward()
  printAccuracy(preds, y)

  for p in weights, bias:
    p.data -= p.grad*lr
    p.grad.zero_()


